# Transfer cached entries from dboeannotation to the XML database (vleserver)

In [1]:
from typing import Dict
from typing import Set
import asyncio
import nest_asyncio

nest_asyncio.apply()

from dboeannotation.services.async_api_token_auth_service import api_token_auth_create
from dboeannotation.services.async_api_service import api_documents_list, api_documents_change
from vleserver.services.async_entries_service import _getDictDictNameEntries, _changeEntries 

## Get the XML data from the cache

See `dboeannotation/api_config.py` for the endpoint used!
Need to log in and get an token to further use the API.

In [2]:
async def getXmlFromCache() -> Dict:
    from dboeannotation.api_config import APIConfig;
    config = APIConfig()
    response = await api_token_auth_create({'username': 'vle', 'password': 'changeme_please'}, config)
    # print(response['token'])
    config.set_access_token(response['token'])
    return await api_documents_list(cache_only=True, page_size=50, api_config_override=config)

In [3]:
cachedData = await getXmlFromCache()
for r in { r["xml_modified_by"] for r in cachedData["results"] }:
    print(r)

mgrossfurtner
testuser
None
bkrautgartner


In [4]:
for by in {result["xml_modified_by"] for result in cachedData["results"]}:
    es_ids = ','.join([result['es_id'] for result in cachedData['results'] if result["xml_modified_by"] == by])
    print(by, es_ids)

mgrossfurtner f250_qdb-d1e10012
testuser f231_qdb-d1e100017
None f231_qdb-d1e100453,f231_qdb-d1e100106,f231_qdb-d1e100424,f231_qdb-d1e1002,f231_qdb-d1e100335,bk_td_2023_05_11_003,bk_td_2023_05_11_002,f231_qdb-d1e100186,f231_qdb-d1e100362,f231_qdb-d1e100152,f231_qdb-d1e100392,f231_qdb-d1e101481,f231_qdb-d1e100302
bkrautgartner f231_qdb-d1e100079,f231_qdb-d1e100056


In [5]:
await api_documents_change(cachedData['results'])

HTTPException: 401  failed with status code: 401